In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.7 MB/s eta 0:00:00


In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.3 MB/s eta 0:00:00


In [4]:
!pip install numpy

In [5]:
!pip install transformers

In [6]:
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

def load_text_from_txt(txt_path):
    with open(txt_path, 'r', encoding='utf-8') as file:
        text = file.read().strip().split('\n')
    return text

def index_text(sentences):
    embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    embeddings = embedder.encode(sentences)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, sentences, embedder

def search(index, embedder, sentences, query, k=5):
    query_embedding = embedder.encode([query])
    D, I = index.search(query_embedding, k)
    results = [sentences[i] for i in I[0]]
    return results

def generate_answer(text_generator, tokenizer, index, embedder, sentences, query, with_context):
    if with_context:
      results = search(index, embedder, sentences, query)
      context = " ".join(results)

      print(f"\n\nContexto:\n {context}")

      messages = [
          {"role": "system", "content": f"Você é um especialista em ferrovias Brasileiras e conhece bem o Regulamento de Operações Ferroviária. Com esse trecho do documento de regulamento: {context}. Encontre a melhor reposta para a perguntas e duvidas das pessoas que atuam nas operações ferroviarias."},
          {"role": "user", "content": query}
      ]
    else:
      messages = [
          {"role": "system", "content": "Você é um especialista em ferrovias Brasileiras e conhece bem o Regulamento de Operações Ferroviária."},
          {"role": "user", "content": query}
      ]

    pipe = pipeline(
        "text-generation",
        model=text_generator,
        tokenizer=tokenizer,
        device=0
    )

    generation_args = {
        "max_new_tokens": 500, # Número máximo de tokens a serem gerados. Em outras palavras, o tamanho da sequência de saída, não incluindo os tokens do prompt
        "return_full_text": False,
        "temperature": 0.1,
        "top_p": 0.3,
        "do_sample": True,
        "output_scores": True
    }

    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

# Carrega e indexa o texto ao iniciar o servidor
txt_path = "rof.txt"
sentences = load_text_from_txt(txt_path)
index, sentences, embedder = index_text(sentences)




/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Inicializa modelo de geração de texto
torch.random.manual_seed(0)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
text_generator = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [9]:
query = "O que significa Linha Tronco"

#answer = generate_answer(text_generator, tokenizer, index, embedder, sentences, query, False)
#print(f"\n\nReposta sem contexto:\n {answer}")

answer = generate_answer(text_generator, tokenizer, index, embedder, sentences, query, True)
print(f"\n\nReposta com contexto:\n {answer}")



Contexto:
 Linha Principal – Linha de circulação que atravessa os pátios e locações, interligando-os.  Linha Tronco – Linha de um sistema ferroviário que, em virtude de suas características de circulação, é de maior importância relativamente às demais.  Linhas Secundárias – Linhas ou desvios adjacentes a uma ou mais linhas principais.  Gabarito de Circulação – área de interferência além da qual o tráfego ferroviário se dá com segurança e sem riscos de abalroamento com obstáculos fixos ou móveis. É medido a partir do eixo da via férrea em relação às laterais da via e a partir do topo do trilho, em relação à altura. A expressão “fora de gabarito de circulação” é utilizada para indicar a posição além da qual o tráfego ferroviário se dá com segurança. Ao contrário a expressão “dentro de gabarito de circulação” é utilizada para indicar a posição interna do gabarito. 74. Trem com Carga Fora de Gabarito ou Excesso Lateral


Reposta com contexto:
  A Linha Tronco é a linha principal de um si